In [2]:
import numpy as np
from datetime import date

n = 10000 # 시뮬레이션 횟수
r = 0.0165 # 이자율
volatility = 0.1778 # 변동성

# 장이 열리는 일자만 고려할 시
#n0 = np.datetime64('2018-03-23') # 최초 기준가격평가일
## datatime.date(2018,03,23)과 동일
#n1 = np.datetime64('2018-09-19') # 1차 조기상환일
#n2 = np.datetime64('2019-03-20') # 2차 조기상환일
#n3 = np.datetime64('2019-09-19') # 3차 조기상환일
#n4 = np.datetime64('2020-03-19') # 4차 조기상환일
#n5 = np.datetime64('2020-09-21') # 5차 조기상환일
#n6 = np.datetime64('2021-03-19') # 만기 상환일
#
#T1 = np.busday_count(n0, n1) # 1차 조기상환일까지 남은 일수, 128
#T2 = np.busday_count(n0, n2) # 2차 조기상환일까지 남은 일수, 258
#T3 = np.busday_count(n0, n3) # 3차 조기상환일까지 남은 일수, 389
#T4 = np.busday_count(n0, n4) # 4차 조기상환일까지 남은 일수, 519
#T5 = np.busday_count(n0, n5) # 5차 조기상환일까지 남은 일수, 651
#T6 = np.busday_count(n0, n6) # 만기일까지 남은 일수, 780

# 365일 다 고려할 시
n0 = date.toordinal(date(2018,3,23)) # 최초 기준가격평가일
n1 = date.toordinal(date(2018,9,19)) # 1차 조기상환일
n2 = date.toordinal(date(2019,3,20)) # 2차 조기상환일
n3 = date.toordinal(date(2019,9,19)) # 3차 조기상환일
n4 = date.toordinal(date(2020,3,19)) # 4차 조기상환일
n5 = date.toordinal(date(2020,9,21)) # 5차 조기상환일
n6 = date.toordinal(date(2021,3,19)) # 만기 조기상환일

T1 = n1-n0 # 1차 조기상환일까지 남은 일수, 180
T2 = n2-n0 # 2차 조기상환일까지 남은 일수, 362
T3 = n3-n0 # 3차 조기상환일까지 남은 일수, 545
T4 = n4-n0 # 4차 조기상환일까지 남은 일수, 727
T5 = n5-n0 # 5차 조기상환일까지 남은 일수, 913
T6 = n6-n0 # 만기일까지 남은 일수, 1092


#oneyar # 1년을 장 개시일 기준으로 할 때
oneyear = 365
total_date = T6 # 780
dt = 1/oneyear # 시간 격자 간격
S = np.zeros((total_date+1, 1)) # 주가벡터 생성(781,1)
S[0] = 100 # 초기 주가
strike_price = S[0] * [0.95, 0.95, 0.95, 0.90, 0.90, 0.85] #(6,)
repay_n = len(strike_price) # 조기상환횟수, 6
coupon_rate = [0.022, 0.044, 0.066, 0.088, 0.11, 0.132] # 조기행사시 받는 쿠폰 이자율
check_day = [T1, T2, T3, T4, T5, T6] # 조기상환일 벡터 

total_payoff = np.zeros((repay_n, 1)) # 전체 payoff 벡터
payment = np.zeros((repay_n, 1)) # 조기상환시 payoff 벡터

facevalue = 10000
kib = 0.65 * S[0] # Knock In Barrier : 65
dummy = 0.132

#%% 조기상환시의 payoff 벡터 생성
for i in range(repay_n):
    payment[i] = facevalue * (1 + coupon_rate[i])
#   payment : array([[10220.],
#                    [10440.],
#                    [10660.],
#                    [10880.],
#                    [11100.],
#                    [11320.]])

#%% Monte-Carlo Simulation을 이용한 ELS Pricing
for i in range(n):
    # 임의의 주가 경로 생성
    for j in range(total_date):
        S[j+1] = S[j]*np.exp((r-0.5*volatility**2)*dt \
                             + volatility*np.sqrt(dt)*np.random.randn(1))
    
    # 조기상환일 체크하여 payoff 결정
    S_checkday = S[check_day] # 해당 조기 상환일의 주가
    payoff = np.zeros((repay_n, 1)) # payoff 벡터 초기화
    
    repay_event = 0 # 조기상환이 되지 않은 상태를 의미함. for문을 빠져나감
    
    for j in range(repay_n):
        if S_checkday[j] >= strike_price[j]: # 만약 해당일의 주가가 행사가격보다 크거나 같으면,
            payoff[j] = payment[j] # 조기상환시 약속한 지불금액이 payoff가 된다.
            repay_event = 1 # 조기상환이 된 상태를 의미함
            break
    
    # 조기상환이 되지 않고 만기까지 온 경우
    if repay_event == 0:
        if min(S) > kib : # Knock-In-barrier 아래로 내려간적이 없으면
            payoff[-1] = facevalue*(1 + dummy) # 액면가에 dummy rate를 곱한 만큼 지불(가장 좋은 상황)
        else:
            payoff[-1] = facevalue*(S[-1]/S[0]) # 액면가에 마지막주가/초기가 만큼 곱해서 지불(가장 안 좋은 상황)
    
    # 한 번 시뮬레이션이 끝날 때 마다 total_payoff 해당 상황 자리에 쌓아줌
    total_payoff = total_payoff + payoff
 
mean_payoff = total_payoff/n #모든 시뮬레이션 payoff의 평균을 구함

#%% ELS Pricing
discount_payoff = np.zeros_like(mean_payoff)
for i in range(repay_n):
    discount_payoff[i] = mean_payoff[i]*np.exp(-r*check_day[i]/oneyear)


Price = sum(discount_payoff) 

In [3]:
print(Price)

[9872.34264275]
